In [1]:
"""Implementations of algorithms for continuous control."""
import functools
from jaxrl_m.typing import *

import jax
import jax.lax as lax
import jax.numpy as jnp
import numpy as np
import optax
from jaxrl_m.common import TrainState, target_update, nonpytree_field
from jaxrl_m.networks import DeterministicPolicy,Policy, Critic, ensemblize

import flax
import flax.linen as nn

from functools import partial



class SACAgent(flax.struct.PyTreeNode):
    rng: PRNGKey
    critic: TrainState
    target_critic: TrainState
    actor: TrainState
    config: dict = nonpytree_field()
    
    
    @jax.jit    
    def reset_critic_optimizer(agent):
    
        new_opt_state = agent.critic.tx.init(agent.critic.params)
        new_critic = agent.critic.replace(opt_state=new_opt_state)
        
        return agent.replace(critic=new_critic)


    #     # update = lambda target : target_update(new_critic,target,agent.config['target_update_rate'])
    #     # no_update = lambda target: target
    #     # new_target_critic = lax.cond(num_update%2==0,update,no_update,agent.target_critic)
        
        
        
    @partial(jax.jit,static_argnames=('num_steps',))  
    def update_critic(agent, transitions: Batch,idxs:jnp.array,num_steps:int):
        
        def one_update(agent, batch: Batch):
                
            new_rng, curr_key, next_key = jax.random.split(agent.rng, 3)

        
            def critic_loss_fn(critic_params):
                next_actions = agent.actor(batch['next_observations'])
                
                next_qs = agent.target_critic(batch['next_observations'], next_actions,True,
                                                    params=None,rngs={'dropout': next_key})
                next_q = jnp.min(next_qs,axis=0)
                target_q = batch['rewards'] + agent.config['discount'] * batch['masks'] * next_q

                qs = agent.critic(batch['observations'], batch['actions'],True,
                                                    params=critic_params,rngs={'dropout': curr_key},
                                                    )
                
                critic_loss = ((target_q-qs)**2).mean()
                
                return critic_loss, {
                    'critic_loss': critic_loss,
                    'q1': qs.mean(),
                }  
            
            new_critic, critic_info = agent.critic.apply_loss_fn(loss_fn=critic_loss_fn, has_aux=True)
            new_target_critic = target_update(agent.critic, agent.target_critic, agent.config['target_update_rate'])
            return agent.replace(rng=new_rng, critic=new_critic, target_critic=new_target_critic)
        
        
        get_batch = lambda transitions,idx : jax.tree_map(lambda x : x[idx],transitions)
        
        agent = jax.lax.fori_loop(0, num_steps, 
                        lambda i, agent: one_update(agent,get_batch(transitions,idxs[i])),
                        agent)
        
        return agent,{}
       
    @jax.jit 
    def train_critics(agent, transitions: Batch,idxs:jnp.array,num_steps:int,R2_history:jnp.ndarray):
        
        rng = jax.random.PRNG(0)
        rngs = jax.random.split(rng,5)
        n_critics = 5
        critic_mask = jnp.zeros((n_critics,))
        opt_mask = jnp.ones(n_critics)
        
    
        if jnp.min(R2_history) < 0 :
                                critic_mask.at[jnp.argmin(R2_history)].set(1)
                                print(f'resetting  {np.argmin(R2_history)}')
                                                                    
        # b_critic_params = reset_critic_vmap(critic_mask,rngs,agent_state.b_critic_params)
        # b_critic_target_params = reset_critic_vmap(critic_mask,rngs,agent_state.b_critic_target_params)
        # b_critic_opt_state = reset_opt_vmap(opt_mask,rngs,agent_state.b_critic_opt_state)
        # b_batch_idxs = self.generate_batch_vmap(rngs,agent_state.buffer_max_size,num_steps,self.critic_batch_size)
        
    

        
    @jax.jit
    def update_actor(agent, batch: Batch):
        new_rng, curr_key, next_key = jax.random.split(agent.rng, 3)

        def actor_loss_fn(actor_params):
            
            actions = agent.actor(batch['observations'], params=actor_params)
            qs = agent.critic(batch['observations'], actions)
            q = qs.mean(axis=0)
            actor_loss = (-q).mean()
            
            return actor_loss, {
                'actor_loss': actor_loss,
              
            }

        new_actor, actor_info = agent.actor.apply_loss_fn(loss_fn=actor_loss_fn, has_aux=True)

        return agent.replace(rng=new_rng,actor=new_actor,), {**actor_info}
        


    @jax.jit
    def sample_actions(agent,observations: np.ndarray) -> jnp.ndarray:
        actions = agent.actor(observations)
       
        return actions
    
 

def create_learner(
                 seed: int,
                 observations: jnp.ndarray,
                 actions: jnp.ndarray,
                 actor_lr: float = 3e-4,
                 critic_lr: float = 3e-4,
                 hidden_dims: Sequence[int] = (256, 256),
                 discount: float = 0.99,
                 tau: float = 0.005,
            **kwargs):

        print('Extra kwargs:', kwargs)

        rng = jax.random.PRNGKey(seed)
        rng, actor_key, critic_key = jax.random.split(rng, 3)

        action_dim = actions.shape[-1]
        actor_def = DeterministicPolicy((64,64), action_dim=action_dim,final_fc_init_scale=1.0)

        actor_params = actor_def.init(actor_key, observations)['params']
        actor = TrainState.create(actor_def, actor_params, tx=optax.adam(learning_rate=actor_lr))
        
        
        critic_def = ensemblize(Critic, num_qs=2,split_rngs={"dropout":True})(hidden_dims)
        critic_params = critic_def.init(critic_key, observations, actions)['params']
        critic = TrainState.create(critic_def, critic_params, tx=optax.adam(learning_rate=critic_lr))
        target_critic = TrainState.create(critic_def, critic_params)        

        # critic_def = Critic(hidden_dims)
        # critics_keys = jax.random.split(critic_key,5)
        # critics_params = jax.vmap(critic_def.init,in_axes=(0,None,None))(critics_keys,observations,actions)
        # tmp = partial(TrainState.create,critic_def,tx=optax.adam(learning_rate=critic_lr))
        # critics = jax.vmap(tmp)(critics_params)
        # target_critics = jax.vmap(TrainState.create,in_axes=(None,0))(critic_def,critics_params)

    
        config = flax.core.FrozenDict(dict(
            discount=discount,
            target_update_rate=tau,    
        ))

        return SACAgent(rng, critic=critic, target_critic=target_critic, actor=actor, config=config)
        #return SACAgent(rng, critic=critics, target_critic=target_critics, actor=actor, config=config)



/home/mahdi/Desktop/supersac/.venv/lib/python3.10/site-packages/jax/_src/api_util.py:229: SyntaxWarning: Jitted function has invalid argnames {'num_batches'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "


In [2]:
from jaxrl_m.dataset import ReplayBuffer
from flax import struct
import chex



@struct.dataclass
class PolicyRollout:
    
    policy_params : chex.Array
    num_rollouts : chex.Array 
    policy_return : chex.Array
    observations : chex.Array
    disc_masks : chex.Array
    

def rollout_policy(agent,env,exploration_rng,
                   replay_buffer,actor_buffer,
                   warmup=False,num_rollouts=5,):
    
    
    actor_buffer.reset()
    obs,_ = env.reset()  
    n_steps,n_rollouts,episode_step,disc,mask = 0,0,0,1.,1.
    max_steps = num_rollouts*1000
    observations,disc_masks,rewards = np.zeros((max_steps,obs.shape[0])),np.zeros((max_steps,)),np.zeros((max_steps,))
    
    
    while n_rollouts < num_rollouts:
        
        if warmup:
            action = env.action_space.sample()
        else:
            exploration_rng, key = jax.random.split(exploration_rng)
            action = agent.sample_actions(obs)
        
        next_obs, reward, done, truncated, info = env.step(action)
        #reward = reward/400
        
        mask = float(not done)
    
        transition = dict(observations=obs,actions=action,
            rewards=reward,masks=mask,next_observations=next_obs,discounts=disc)
        
        
        replay_buffer.add_transition(transition)
        actor_buffer.add_transition(transition)
    
        observations[1000*n_rollouts+episode_step] = obs
        disc_masks[1000*n_rollouts+episode_step] = disc
        rewards[1000*n_rollouts+episode_step] = reward
        
        obs = next_obs
        disc *= (0.99*mask)
        episode_step += 1
        n_steps += 1
        
        if (done or truncated) :
            
            #exploration_metrics = {f'exploration/{k}': v for k, v in flatten(info).items()}
            obs,_= env.reset()
            n_rollouts += 1
            episode_step = 0
            disc,mask = 1.,1.

    policy_return = (disc_masks*rewards).sum()/num_rollouts
    policy_rollout = PolicyRollout(policy_params=agent.actor.params,
                                   policy_return=policy_return,
                                   observations=observations,
                                   disc_masks=disc_masks,
                                    num_rollouts=num_rollouts)
    
    return replay_buffer,actor_buffer,policy_rollout,policy_return,n_steps

In [3]:




def f(anc_agent,obs,actor):

    actions = anc_agent.actor(obs, params=actor)
    qs = anc_agent.critic(obs, actions,params=anc_agent.target_critic.params)
    #q = jnp.minimum(q1, q2)
    q = qs.mean(axis=0)
    
    return q
    

@jax.jit
def estimate_return(anc_agent,anc_return,acq_rollout:PolicyRollout,):
    
    acq_obs = acq_rollout.observations
    acq_masks = acq_rollout.disc_masks
  
    acq_actor = acq_rollout.policy_params
    acq_return = acq_rollout.policy_return
    
    anc_actor = anc_agent.actor.params
    
    acq_q = f(anc_agent,acq_obs,acq_actor)
    anc_q = f(anc_agent,acq_obs,anc_actor)
    
    adv = ((acq_q - anc_q)*acq_masks).sum()/5
    acq_return_pred = anc_return + adv
  
    
    return acq_return_pred,acq_return


def evaluate_critic(anc_agent,anc_return,policy_rollouts):

    y_pred,y= [],[]
    for policy_rollout in policy_rollouts:
        
        acq_return_pred,acq_return = estimate_return(anc_agent,anc_return,policy_rollout)
        y_pred.append(acq_return_pred),y.append(acq_return)
        
    y_pred,y = np.array(y_pred),np.array(y)
    a2 = jnp.clip(((y-y_pred)**2),a_min=1e-4).sum()
    b2=((y-y.mean())**2).sum()
    R2 = 1-(a2/b2)  
    
    return R2


In [4]:
import os
from functools import partial
import numpy as np
import jax
import tqdm
import gymnasium as gym


from jaxrl_m.wandb import setup_wandb, default_wandb_config, get_flag_dict
import wandb
from jaxrl_m.evaluation import supply_rng, evaluate, flatten, EpisodeMonitor
from jaxrl_m.dataset import ReplayBuffer
from collections import deque


env_name='InvertedPendulum-v4'
seed=np.random.choice(1000000)
eval_episodes=10
batch_size = 256
max_steps = int(1e6)
start_steps = 50000                   
log_interval = 5000
#eval_interval = 10000

wandb_config = default_wandb_config()
wandb_config.update({
    'project': 'd4rl_test',
    'group': 'sac_test',
    'name': 'sac_{env_name}',
})


env = EpisodeMonitor(gym.make(env_name))
eval_env = EpisodeMonitor(gym.make(env_name))
setup_wandb({"bonjour":1})

example_transition = dict(
    observations=env.observation_space.sample(),
    actions=env.action_space.sample(),
    rewards=0.0,
    masks=1.0,
    next_observations=env.observation_space.sample(),
    discounts=1.0,
)

replay_buffer = ReplayBuffer.create(example_transition, size=int(1e6))
actor_buffer = ReplayBuffer.create(example_transition, size=int(5e3))

agent = create_learner(seed,
                example_transition['observations'][None],
                example_transition['actions'][None],
                max_steps=max_steps,
                #**FLAGS.config
                )

exploration_metrics = dict()
obs,info = env.reset()    
exploration_rng = jax.random.PRNGKey(0)
i = 0
unlogged_steps = 0
policy_rollouts = deque([], maxlen=10)
with tqdm.tqdm(total=max_steps) as pbar:
    
    while i < max_steps:
    
        replay_buffer,actor_buffer,policy_rollout,policy_return,num_steps = rollout_policy(agent,env,exploration_rng,
                   replay_buffer,actor_buffer,
                   warmup=(i < start_steps))
        policy_rollouts.append(policy_rollout)
        unlogged_steps += num_steps
        i+=num_steps
        pbar.update(num_steps)
        
            
        if replay_buffer.size > start_steps:
        
         
            transitions = replay_buffer.get_all()
            idxs = jax.random.choice(a=replay_buffer.size, shape=(10000,256), replace=True,key=jax.random.PRNGKey(0))
            agent.reset_critic_optimizer()
            agent, critic_update_info = agent.update_critic(transitions,idxs,10000)
            R2 = evaluate_critic(agent,policy_rollouts[-1].policy_return,policy_rollouts)

            
            actor_batch = actor_buffer.get_all()      
            agent, actor_update_info = agent.update_actor(actor_batch)    
            
            
            update_info = {**critic_update_info, **actor_update_info, 'R2_validation': R2}
            
            
            if unlogged_steps > log_interval:
                exploration_metrics = {f'exploration/disc_return': policy_return}
                wandb.log(exploration_metrics, step=int(i),commit=False)
                train_metrics = {f'training/{k}': v for k, v in update_info.items()}
                wandb.log(train_metrics, step=int(i),commit=False)
                #wandb.log(exploration_metrics, step=i)
                policy_fn = agent.actor
                eval_info = evaluate(policy_fn, eval_env, num_episodes=eval_episodes)
                eval_metrics = {f'evaluation/{k}': v for k, v in eval_info.items()}
                print('evaluating')
                wandb.log(eval_metrics, step=int(i),commit=True)
                unlogged_steps = 0


2023-12-15 15:35:05.176811: W external/xla/xla/service/gpu/nvptx_compiler.cc:679] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.103). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin


Extra kwargs: {'max_steps': 1000000}


  5%|▍         | 49375/1000000 [00:02<00:39, 24009.71it/s]

evaluating


  6%|▌         | 55600/1000000 [01:06<2:00:05, 131.07it/s]

...

evaluating


 18%|█▊        | 177571/1000000 [09:40<41:43, 328.54it/s]  

evaluating


 18%|█▊        | 181929/1000000 [09:54<44:32, 306.13it/s]  


KeyboardInterrupt: 